In [3]:
import bayesflow as bf
import tensorflow_probability as tfp
import tensorflow as tf
import numpy as np
import torch
from torch.distributions import Normal, Independent,InverseGamma

/opt/anaconda3/envs/bayesflow/lib/python3.11/site-packages/bayesflow/trainers.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


here we try to simulate from discrete distribution. There is no errors during the training.But in theory, because gradient descent requires derivatives, it's problematic. The use of discrete has not been studied

In [4]:

def prior_fun(batch_size=1):
    # Discrete 
    mean, stddev = 10.0, 1.0
    miu_dist = tfp.distributions.Bernoulli(0.5)
    miu_samples = tf.cast(miu_dist.sample(batch_size), dtype=tf.float32)

    # InverseGamma(alpha, beta)
    alpha, beta = 2.0, 1.0
    sigma2_dist = tfp.distributions.InverseGamma(concentration=alpha, scale=beta)
    sigma2_samples = tf.cast(sigma2_dist.sample(batch_size), dtype=tf.float32)
    # (batch_size, 2) 
    return tf.concat([miu_samples[:, None], sigma2_samples[:, None]], axis=-1)



def likelihood_fun(params, n_obs=1000):
    miu = params[:, 0]  # miu
    sigma2 = params[:, 1]  # sigma^2
    return tf.random.normal(
        shape=(params.shape[0], n_obs, 1),
        mean=miu[:, None],
        stddev=tf.sqrt(sigma2)[:, None]
    )

In [5]:

prior = bf.simulation.Prior(prior_fun=prior_fun)
simulator = bf.simulation.Simulator(simulator_fun=likelihood_fun)
model = bf.simulation.GenerativeModel(prior=prior, simulator=simulator)

summary_net = bf.networks.DeepSet(summary_dim=10)

inference_net = bf.networks.InvertibleNetwork(
    num_params=2,
    num_coupling_layers=4,
)

amortizer = bf.amortizers.AmortizedPosterior(inference_net, summary_net)



INFO:root:Performing 2 pilot runs with the anonymous model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 1, 2)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 1, 1000, 1)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:No optional simulation batchable context provided.


In [6]:
# Define the Trainer
trainer = bf.trainers.Trainer(amortizer=amortizer, generative_model=model)

# Perform a single model call (optional, to verify the model is working correctly)
_ = model(batch_size=32)

INFO:root:Performing a consistency check with provided components...
INFO:root:Done.


In [ ]:
# 开始在线训练
history = trainer.train_online(
    epochs=5,               # 10 epoch
    iterations_per_epoch=500,  
    batch_size=32,        
    validation_sims=200    
)